In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image
from transformers import AutoProcessor, CLIPVisionModel
import glob
import numpy as np
import torch

# Initialize the model and processor
model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")



# Paths to the Google Drive folders
# folder1 = "./retrieved_images8"
# folder1 = "./retrieved_images9"
folder1 = "./retrieved_images1002"


# folder2 = "./Adapter1_generated_images8"
# folder2 = "./Adapter1_generated_images9"
# folder2 = "./Adapter1_generated_images1002"

# folder2 = "./DB1_generated_images8"
# folder2 = "./DB1_generated_images9"
# folder2 = "./DB1_generated_images1002"


# folder2 = "./Adapter3_generated_images8"
# folder2 = "./Adapter3_generated_images9"
# folder2 = "./Adapter3_generated_images1002"

# folder2 = "./t8_new"
# folder2 = "./t9_new"
folder2 = "./t1002_new"




# Load image paths
images_folder1 = glob.glob(f"{folder1}/*.png")  # Adjust extension if needed
images_folder2 = glob.glob(f"{folder2}/*.jpg") #or png

# Ensure we have images
if len(images_folder1) == 0 or len(images_folder2) == 0:
    raise ValueError("One of the folders is empty. Please check the paths.")

# Compute pairwise cosine similarities
pairwise_similarities = []

for img1_path in images_folder1:
    image1 = Image.open(img1_path).convert("RGB")
    inputs1 = processor(images=image1, return_tensors="pt")
    outputs1 = model(**inputs1)
    pooled_output1 = outputs1.pooler_output.detach().numpy()[0]

    for img2_path in images_folder2:
        image2 = Image.open(img2_path).convert("RGB")
        inputs2 = processor(images=image2, return_tensors="pt")
        outputs2 = model(**inputs2)
        pooled_output2 = outputs2.pooler_output.detach().numpy()[0]

        # Compute cosine similarity
        dot_product = np.dot(pooled_output1, pooled_output2)
        magnitude1 = np.linalg.norm(pooled_output1)
        magnitude2 = np.linalg.norm(pooled_output2)

        cosine_sim = dot_product / (magnitude1 * magnitude2)
        pairwise_similarities.append(cosine_sim)

# Calculate the sum and mean of cosine similarities
sum_similarity = np.sum(pairwise_similarities)
mean_similarity = np.mean(pairwise_similarities)

# Store and print results
print(folder1)
print(folder2)
print("Pairwise Similarities:", pairwise_similarities)
print(f"Sum of Similarities: {sum_similarity}")
print(f"Mean of Similarities: {mean_similarity}")
